## Download and Install Requirements

In [ ]:
!pip install -r requirements.txt

## Download Cropped APS Images and Raw EMG Data

In [ ]:
import os
from os import path
import wget
import zipfile
from zipfile import ZipFile


relax21_cropped_aps = 'https://zenodo.org/record/3663616/files/relax21_cropped_aps.zip'
relax21_raw_emg = 'https://zenodo.org/record/3663616/files/relax21_raw_emg.zip'
data_directory = 'data'
data_downloaded = False
if not os.path.isdir(data_directory):
    os.mkdir(data_directory)

if not [f for f in os.listdir(data_directory) if not f.startswith('.')] == []:
    data_downloaded = True

if data_downloaded == False:
    try:
        if not path.isfile('relax21_cropped_aps.zip'):
            wget.download(relax21_cropped_aps, 'relax21_cropped_aps.zip')
     
        if not path.isfile('relax21_raw_emg.zip'):
            wget.download(relax21_raw_emg, 'relax21_raw_emg.zip')

        relax21_cropped_aps_zip = ZipFile('relax21_cropped_aps.zip', 'r')
        relax21_cropped_aps_zip.extractall('data')
        relax21_cropped_aps_zip.close()
        relax21_raw_emg_zip = ZipFile('relax21_raw_emg.zip', 'r')
        relax21_raw_emg_zip.extractall('data')
        relax21_raw_emg_zip.close() 
    except Exception as e:
        print('Exception: %s raised. Failed to setup environment.' % e)

## Load Cropped APS Images and Raw EMG Data and Extract Input Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils import Person, analyze
from utils import Person, analyze, corrections
import torch


classes = ['pinky', 'elle', 'yo', 'index', 'thumb']
classes_dict = {'pinky': 0, 'elle': 1, 'yo': 2, 'index': 3, 'thumb': 4}
classes_inv = {v: k for k, v in classes_dict.items()}
crop_path = 'data/img_cropped/'
emg_dir = 'data/EMG/'

def load_emg_dataset():
    subjects = {}
    names = sorted([name for name in os.listdir(emg_dir) if "emg" in name])
    for name in names:
        _emg = np.load(emg_dir + '{}'.format(name)).astype('float32')
        _ann = np.concatenate([np.array(['none']), np.load(emg_dir + '{}'.format(name.replace("emg", "ann")))[:-1]])
        subjects["_".join(name.split("_")[:2])] = Person(name.split("_")[0], _emg, _ann, classes=classes)

    print("Data Loaded! {} Sessions".format(len(subjects.keys())))
    for name, data in subjects.items():
        for _class in classes:
            _annotation = np.float32(data.ann == _class)
            derivative = np.diff(_annotation) / 1.0
            begins = np.where(derivative == 1)[0]
            ends = np.where(derivative == -1)[0]
            for b, e in zip(begins, ends):
                _trials = data.emg[b:e]
                data.trials[_class].append(_trials)
                data.begs[_class].append(b)
                data.ends[_class].append(e)
                
    print("Done sorting trials!")
    X_EMG = []
    Y_EMG = []
    SUB_EMG = []
    SES_EMG = []
    TRI_EMG = []
    for name, data in subjects.items():
        for gesture in classes:
            for trial in range(5):
                X_EMG.append(data.trials[gesture][trial])
                Y_EMG.append(classes_dict[gesture])
                SUB_EMG.append(int(name[7:9]))
                SES_EMG.append(int(name[17:19]))
                TRI_EMG.append(trial)

    X_EMG = np.array(X_EMG)
    Y_EMG = np.array(Y_EMG)
    SUB_EMG = np.array(SUB_EMG)
    SES_EMG = np.array(SES_EMG)
    TRI_EMG = np.array(TRI_EMG)
    return X_EMG, SUB_EMG, SES_EMG, TRI_EMG, Y_EMG

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.144])

def load_img_dataset():
    all_frames = [i for i in sorted(os.listdir(crop_path)) if 'tiff' in i]
    Y_IMG = np.array([classes_dict[i.split('_')[2]] for i in sorted(os.listdir(crop_path)) if 'tiff' in i])
    SUB_IMG = np.array([int(i[7:9]) for i in sorted(os.listdir(crop_path)) if 'tiff' in i])
    SES_IMG = np.array([int(i[17:19]) for i in sorted(os.listdir(crop_path)) if 'tiff' in i])
    TRI_IMG = np.array([int(i.split('_')[3]) for i in sorted(os.listdir(crop_path)) if 'tiff' in i])
    IDX_IMG = np.array([int(i.split('_')[4].split('.')[0]) for i in sorted(os.listdir(crop_path)) if 'tiff' in i])
    IDX_IMG = np.array(IDX_IMG)
    Y_IMG = np.array(Y_IMG)
    SUB_IMG = np.array(SUB_IMG)
    SES_IMG = np.array(SES_IMG)
    TRI_IMG = np.array(TRI_IMG)
    X_IMG = np.array([rgb2gray(plt.imread(crop_path + f))[::-1] for f in all_frames])
    X_IMG = np.array(X_IMG)
    return X_IMG, SUB_IMG, SES_IMG, TRI_IMG, Y_IMG

def sync_datasets(EMG, IMG, frame_len=0.2, frame_step=0.1):
    X_EMG, SUB_EMG, SES_EMG, TRI_EMG, Y_EMG = EMG
    X_IMG, SUB_IMG, SES_IMG, TRI_IMG, Y_IMG = IMG
    F_SUB = []
    F_SESS = []
    F_Y = []
    F_IMG = []
    F_EMG = []
    for subject in range(1, 22):
        for session in range(1, 4):
            for gesture in range(5):
                for trial in range(5):
                    fs = corrections['subject{:02}_session0{}'.format(subject, session)]['fs']
                    idx_emg = np.logical_and.reduce([SUB_EMG == subject,
                                                     SES_EMG == session,
                                                     TRI_EMG == trial,
                                                     Y_EMG == gesture])
                    idx_img = np.logical_and.reduce([SUB_IMG == subject,
                                                     SES_IMG == session,
                                                     TRI_IMG == trial,
                                                     Y_IMG == gesture])
                    
                    e = X_EMG[idx_emg][0]
                    f = X_IMG[idx_img]
                    mav = analyze(e, fs=fs, frame_len=frame_len, frame_step=frame_step, feat='MSV', preprocess=False)
                    rms = analyze(e, fs=fs, frame_len=frame_len, frame_step=frame_step, feat='RMS', preprocess=False)
                    a = np.concatenate([mav, rms], 1)
                    mapping = np.arange(len(a)) * len(f) // len(a)
                    F_EMG.append(a)
                    F_IMG.append(np.stack(f[mapping])) # F_IMG = [np.stack(f[mapping])]
                    F_SUB.append(np.ones((len(mapping))) * subject)
                    F_SESS.append(np.ones((len(mapping))) * session)
                    F_Y.append(np.ones((len(mapping))) * gesture)

    F_EMG = np.vstack(F_EMG)
    F_IMG = np.vstack(F_IMG)
    F_SUB = np.hstack(F_SUB)
    F_SESS = np.hstack(F_SESS)
    F_Y = np.hstack(F_Y)
    return F_EMG, F_IMG, F_SUB, F_SESS, F_Y

def create_sets(F_EMG, F_IMG, F_SESS, F_Y, test_ses=1):
    x_emg_train = F_EMG[F_SESS != test_ses].astype('float32')
    x_emg_test = F_EMG[F_SESS == test_ses].astype('float32')
    x_img_train = F_IMG[F_SESS != test_ses].astype('float32')
    x_img_test = F_IMG[F_SESS == test_ses].astype('float32')
    y_train = F_Y[F_SESS != test_ses]
    y_test = F_Y[F_SESS == test_ses]
    return x_emg_train, x_img_train, y_train, x_emg_test, x_img_test, y_test

def preprocess(x_emg_train, x_img_train, x_emg_test, x_img_test):
    data_max = np.max(x_img_train)
    data_min = np.min(x_img_train)
    for i in range(len(x_img_train)):
        x_img_train[i] = (x_img_train[i] - data_min) / (data_max - data_min)
        
    for i in range(len(x_img_test)):
        x_img_test[i] = (x_img_test[i] - data_min) / (data_max - data_min)

    data_mean = np.mean(x_img_train)
    data_std = np.std(x_img_train) + 1e-15
    x_img_train -= data_mean
    x_img_train /= data_std
    x_img_test -= data_mean
    x_img_test /= data_std
    data_max = np.max(x_emg_train)
    data_min = np.min(x_emg_train)
    for i in range(len(x_emg_train)):
        x_emg_train[i] = (x_emg_train[i] - data_min) / (data_max - data_min)
        
    for i in range(len(x_emg_test)):
        x_emg_test[i] = (x_emg_test[i] - data_min) / (data_max - data_min)

    data_mean = np.mean(x_emg_train)
    data_std = np.std(x_emg_train) + 1e-15
    x_emg_train -= data_mean
    x_emg_train /= data_std
    x_emg_test -= data_mean
    x_emg_test /= data_std
    return x_emg_train, x_img_train, x_emg_test, x_img_test

data_dir = 'fold_data'
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
    EMG = load_emg_dataset()
    IMG = load_img_dataset()
    frame_len = 0.2
    for test_ses in [1, 2, 3]:
        print(test_ses)
        F_EMG, F_IMG, F_SUB, F_SESS, F_Y = sync_datasets(EMG, IMG, frame_len=frame_len, frame_step=frame_len / 2)
        x_emg_train, x_img_train, y_train, x_emg_test, x_img_test, y_test = create_sets(F_EMG, F_IMG, F_SESS, F_Y, test_ses=test_ses)
        x_emg_train, x_img_train, x_emg_test, x_img_test = preprocess(x_emg_train, x_img_train, x_emg_test, x_img_test)
        torch.save(torch.tensor(x_emg_train), '%s/fold_%d_x_emg_train.pt' % (data_dir, test_ses - 1)) # EMG training set
        torch.save(torch.tensor(x_emg_test), '%s/fold_%d_x_emg_test.pt' % (data_dir, test_ses - 1)) # EMG test set
        torch.save(torch.tensor(x_img_train), '%s/fold_%d_x_aps_train.pt' % (data_dir, test_ses - 1)) # APS training set
        torch.save(torch.tensor(x_img_test), '%s/fold_%d_x_aps_test.pt' % (data_dir, test_ses - 1)) # APS test set
        torch.save(torch.tensor(y_train), '%s/fold_%d_y_train.pt' % (data_dir, test_ses - 1)) # Training set labels
        torch.save(torch.tensor(y_test), '%s/fold_%d_y_test.pt' % (data_dir, test_ses - 1)) # Test set labels
        del x_emg_train
        del x_emg_test
        del y_train
        del y_test
        # Fused APS-EMG training set
        x_img_train_a = x_img_train[:, ::2, ::2]
        x_img_train_b = x_img_train[:, 1::2, ::2]
        x_img_train_c = x_img_train[:, ::2, 1::2]
        x_img_train_d = x_img_train[:, 1::2, 1::2]
        concat_inp_train = [x_img_train_a.reshape(-1, 400), 
                            x_img_train_b.reshape(-1, 400), 
                            x_img_train_c.reshape(-1, 400), 
                            x_img_train_d.reshape(-1, 400)]
        torch.save(torch.tensor(concat_inp_train), '%s/fold_%d_x_concat_train.pt' % (data_dir, test_ses - 1))
        del x_img_train_a
        del x_img_train_b
        del x_img_train_c
        del x_img_train_d
        del concat_inp_train
        # Fused APS-EMG test set
        x_img_test_a = x_img_test[:, ::2, ::2]
        x_img_test_b = x_img_test[:, 1::2, ::2]
        x_img_test_c = x_img_test[:, ::2, 1::2]
        x_img_test_d = x_img_test[:, 1::2, 1::2]
        concat_inp_test = [x_img_test_a.reshape(-1, 400), 
                           x_img_test_b.reshape(-1, 400), 
                           x_img_test_c.reshape(-1, 400), 
                           x_img_test_d.reshape(-1, 400)]
        torch.save(torch.tensor(concat_inp_test), '%s/fold_%d_x_concat_test.pt' % (data_dir, test_ses - 1)) 
        del x_img_test_a
        del x_img_test_b
        del x_img_test_c
        del x_img_test_d
        del concat_inp_test
        del x_img_train
        del x_img_test

## Define the MLP and CNN Network Architectures

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class MLP_APS_Net(nn.Module):  
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(400, 210)
        self.drop1 = nn.Dropout(0.3, inplace=True)
        self.fc2 = nn.Linear(210, 5)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        return F.softmax(self.fc2(x), dim=1)
    
class MLP_EMG_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 230)
        self.fc2 = nn.Linear(230, 5)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)    
    
class MLP_Fused_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        
    def forward(self, x):
        x = self.fc1(x)
        return F.softmax(x, dim=1)

class Conv_APS_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3)
        self.drop1 = nn.Dropout(0.25, inplace=True)
        self.fc1 = nn.Linear(1152, 512)
        self.drop2 = nn.Dropout(0.5, inplace=True)
        self.fc2 = nn.Linear(512, 5)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)) ,2)
        x = F.relu(self.conv3(x))
        x = self.drop1(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.drop2(x)
        return F.softmax(self.fc2(x), dim=1)    
    
class Conv_EMG_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(16, 128)
        self.drop1 = nn.Dropout(0.5, inplace=True)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 5)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.drop1(x)
        x = F.relu(self.fc2(x))
        return F.softmax(self.fc3(x))
    
class Conv_Fused_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 5)
        
    def forward(self, x):
        x = self.fc1(x)
        return F.softmax(x, dim=1)

## Shared Training Functions

In [ ]:
from torch.utils.data import Dataset, DataLoader
import copy


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class APS(Dataset):
    def __init__(self, data, label, expand_dim=True):
        self.data = data
        self.label = label
        self.expand_dim = expand_dim

    def __getitem__(self, index):
        if self.expand_dim:
            return (torch.unsqueeze(self.data[index], 0), self.label[index].long())
        else:
            return (self.data[index], self.label[index].long())

    def __len__(self):
        return self.label.numel()

def init_weights(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        nn.init.zeros_(m.bias)

def test(model, test_loader):
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):        
        output = model(data.to(device)) 
        pred = output.data.max(1)[1]
        correct += pred.eq(target.to(device).data.view_as(pred)).cpu().sum()

    return 100. * float(correct) / float(len(test_loader.dataset))

def train(model, epochs, fold, train_loader, test_loader, return_model=False, id=None):
    best_accuracy = 0
    for epoch in range(0, epochs):
        print('Fold: [%d] Epoch: [%d]\t' % (fold, epoch + 1), end='')
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model(data.to(device))
            loss = criterion(output, target.to(device))
            loss.backward()
            optimizer.step()

        accuracy = test(net, test_loader)
        print('%2.2f%%' % accuracy)
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = copy.deepcopy(model)
            if id is not None:
                torch.save(net, 'fold_%d_%s.pt' % (fold, id))
            
    if return_model:
        return best_accuracy, best_model
    else:
        return best_accuracy

## Convert all Networks from *Baseline.ipynb*

In [ ]:
!pip install memtorch

In [ ]:
import memtorch
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities


def det_network_type(network_id):
    if 'MLP' in network_id:
        return 'MLP'
    elif 'Conv' in network_id:
        return 'Conv'
        
def det_input_type(network_id):
    if 'APS' in network_id:
        return 'APS'
    elif 'EMG' in network_id:
        return 'EMG'
    elif 'Fused' in network_id:
        return 'Fused'

batch_size = 32
reference_memristor = memtorch.bh.memristor.VTEAM
# Pt/Hf/Ti device parameters
reference_memristor_params = {'time_series_resolution': 1e-6,
                              'alpha_off': 1,
                              'alpha_on': 3,
                              'v_off': 0.5,
                              'v_on': -0.53,
                              'r_off': 2.5e3,
                              'r_on': 100,
                              'k_off': 4.03e-8,
                              'k_on': -80,
                              'd': 10e-9,
                              'x_on': 0,
                              'x_off': 10e-9}

df = pd.DataFrame(columns=['network_type', 'input', 'fold', 'test_set_accuracy'])     
for fold in range(3):
    y_test = torch.load('fold_data/fold_%d_y_test.pt' % fold)
    patched_networks = []
    for model in range(4):
        net = torch.load('fold_%d_MLP_APS_Net_Model_%d.pt' % (fold, model))
        patched_networks.append(patch_model(net,
                                          memristor_model=reference_memristor,
                                          memristor_model_params=reference_memristor_params,
                                          module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                                          mapping_routine=naive_map,
                                          transistor=True,
                                          programming_routine=None))
        patched_networks[model].tune_()
    
    concat_test_output = torch.zeros(y_test.numel(), 5).to(device) 
    for network_idx, network in enumerate(patched_networks):
        inputs = torch.load('fold_data/fold_%d_x_concat_test.pt' % (fold))[network_idx]
        for input_idx, input_ in enumerate(inputs):
            concat_test_output[input_idx, :] += network(torch.tensor(input_).unsqueeze(0).to(device)).squeeze()
            
            
    concat_test_output_ = torch.argmax(concat_test_output, dim=1)
    correct = concat_test_output_.eq(y_test.to(device).data.view_as(concat_test_output_)).cpu().sum()
    test_set_accuracy = 100. * float(correct) / float(y_test.numel())
    df = df.append({'network_type': 'MLP', 'input': 'APS', 'fold': fold, 'test_set_accuracy': test_set_accuracy}, ignore_index=True)
    
networks = {}
networks['MLP_EMG_Net.pt'] = [MLP_EMG_Net, 'x_emg_train', 'x_emg_test', False]
networks['MLP_Fused_Net.pt'] = [MLP_Fused_Net, 'x_fused_train_mlp', 'x_fused_test_mlp', False]
networks['Conv_APS_Net.pt'] = [Conv_APS_Net, 'x_aps_train', 'x_aps_test', True]
networks['Conv_EMG_Net.pt'] = [Conv_EMG_Net, 'x_emg_train', 'x_emg_test', False]
networks['Conv_Fused_Net.pt'] = [Conv_Fused_Net, 'x_fused_train_conv', 'x_fused_test_conv', False]
for fold in range(3):
    y_test = torch.load('fold_data/fold_%d_y_test.pt' % fold)
    for network in networks:
        net = torch.load('fold_%d_%s' % (fold, network))
        patched_net = patch_model(net,
                                  memristor_model=reference_memristor,
                                  memristor_model_params=reference_memristor_params,
                                  module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                                  mapping_routine=naive_map,
                                  transistor=True,
                                  programming_routine=None)
        patched_net.tune_()
        test_loader = DataLoader(APS(torch.load('fold_data/fold_%d_%s.pt' % (fold, networks[network][2])), 
                                 y_test, 
                                 expand_dim=networks[network][3]), batch_size=batch_size, shuffle=False)
        test_set_accuracy = test(patched_net, test_loader)
        df = df.append({'network_type': det_network_type(network), 'input': det_input_type(network), 'fold': fold, 'test_set_accuracy': test_set_accuracy}, ignore_index=True)

df.to_csv('Converted.csv', index=False)  
df = df.sort_values(['network_type','input', 'fold'])
frame = df.groupby(['network_type', 'input'])['test_set_accuracy']
df['test_set_accuracy_mean'] = frame.transform(np.mean)
df['test_set_accuracy_std'] = frame.transform(np.std)
df = df.drop(columns=['test_set_accuracy'])
df = df.drop_duplicates(subset=['network_type', 'input', 'test_set_accuracy_mean', 'test_set_accuracy_std'], keep='first')
df = df.drop(columns=['fold'])
print(df)

## Device-device Variance Investigation

In [ ]:
import memtorch
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities


reference_memristor = memtorch.bh.memristor.VTEAM
df = pd.DataFrame(columns=['network_type', 'input', 'sigma', 'fold', 'test_set_accuracy']) 
sigma_values = np.linspace(0, 500, 21)

for fold in range(3):
    y_test = torch.load('fold_data/fold_%d_y_test.pt' % fold)
    for sigma in sigma_values:
        patched_networks = []
        for model in range(4):
            net = torch.load('fold_%d_MLP_APS_Net_Model_%d.pt' % (fold, model))
            reference_memristor_params = {'time_series_resolution': 1e-6,
                                          'alpha_off': 1,
                                          'alpha_on': 3,
                                          'v_off': 0.5,
                                          'v_on': -0.53,
                                          'r_off': memtorch.bh.StochasticParameter(2.5e3, std=sigma*2, min=1),
                                          'r_on': memtorch.bh.StochasticParameter(100, std=sigma, min=1),
                                          'k_off': 4.03e-8,
                                          'k_on': -80,
                                          'd': 10e-9,
                                          'x_on': 0,
                                          'x_off': 10e-9}
            patched_networks.append(patch_model(net,
                                              memristor_model=reference_memristor,
                                              memristor_model_params=reference_memristor_params,
                                              module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                                              mapping_routine=naive_map,
                                              transistor=True,
                                              programming_routine=None))
            patched_networks[model].tune_()
            
        concat_test_output = torch.zeros(y_test.numel(), 5).to(device) 
        for network_idx, network in enumerate(patched_networks):
            inputs = torch.load('fold_data/fold_%d_x_concat_test.pt' % (fold))[network_idx]
            for input_idx, input_ in enumerate(inputs):
                concat_test_output[input_idx, :] += network(torch.tensor(input_).unsqueeze(0).to(device)).squeeze()
                    
        concat_test_output_ = torch.argmax(concat_test_output, dim=1)
        correct = concat_test_output_.eq(y_test.to(device).data.view_as(concat_test_output_)).cpu().sum()
        test_set_accuracy = 100. * float(correct) / float(y_test.numel())
        df = df.append({'network_type': 'MLP', 'input': 'APS', 'sigma': sigma, 'fold': fold, 'test_set_accuracy': test_set_accuracy}, ignore_index=True)

networks = {}
networks['MLP_EMG_Net.pt'] = [MLP_EMG_Net, 'x_emg_train', 'x_emg_test', False]
networks['MLP_Fused_Net.pt'] = [MLP_Fused_Net, 'x_fused_train_mlp', 'x_fused_test_mlp', False]
networks['Conv_APS_Net.pt'] = [Conv_APS_Net, 'x_aps_train', 'x_aps_test', True]
networks['Conv_EMG_Net.pt'] = [Conv_EMG_Net, 'x_emg_train', 'x_emg_test', False]
networks['Conv_Fused_Net.pt'] = [Conv_Fused_Net, 'x_fused_train_conv', 'x_fused_test_conv', False]
for sigma in sigma_values:
    for fold in range(3):
        y_test = torch.load('fold_data/fold_%d_y_test.pt' % fold)
        for network in networks:
            net = torch.load('fold_%d_%s' % (fold, network))
            reference_memristor_params = {'time_series_resolution': 1e-6,
                                          'alpha_off': 1,
                                          'alpha_on': 3,
                                          'v_off': 0.5,
                                          'v_on': -0.53,
                                          'r_off': memtorch.bh.StochasticParameter(2.5e3, std=sigma*2, min=1),
                                          'r_on': memtorch.bh.StochasticParameter(100, std=sigma, min=1),
                                          'k_off': 4.03e-8,
                                          'k_on': -80,
                                          'd': 10e-9,
                                          'x_on': 0,
                                          'x_off': 10e-9}
            patched_net = patch_model(net,
                                      memristor_model=reference_memristor,
                                      memristor_model_params=reference_memristor_params,
                                      module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                                      mapping_routine=naive_map,
                                      transistor=True,
                                      programming_routine=None)
            patched_net.tune_()
            test_loader = DataLoader(APS(torch.load('fold_data/fold_%d_%s.pt' % (fold, networks[network][2])), 
                                     y_test, 
                                     expand_dim=networks[network][3]), batch_size=batch_size, shuffle=False)
            test_set_accuracy = test(patched_net, test_loader)
            df = df.append({'network_type': det_network_type(network), 'input': det_input_type(network), 'sigma': sigma, 'fold': fold, 'test_set_accuracy': test_set_accuracy}, ignore_index=True)

df.to_csv('Variability.csv', index=False)
df = df.sort_values(['network_type','input', 'sigma', 'fold'])
df['test_set_accuracy'] = pd.to_numeric(df['test_set_accuracy'])
frame = df.groupby(['network_type', 'input', 'sigma'])['test_set_accuracy']
df['test_set_accuracy_mean'] = frame.transform(np.mean)
df['test_set_accuracy_std'] = frame.transform(np.std)
df = df.drop(columns=['test_set_accuracy'])
df = df.drop_duplicates(subset=['network_type', 'input', 'sigma', 'test_set_accuracy_mean', 'test_set_accuracy_std'], keep='first')
df = df.drop(columns=['fold'])
print(df)

## Device-device Variance and Finite Conductance State Investigation

In [ ]:
import memtorch
from memtorch.mn.Module import patch_model
from memtorch.map.Parameter import naive_map
from memtorch.bh.crossbar.Program import naive_program
from memtorch.bh.nonideality.NonIdeality import apply_nonidealities


reference_memristor = memtorch.bh.memristor.VTEAM
df = pd.DataFrame(columns=['network_type', 'input', 'finite_states', 'sigma', 'fold', 'test_set_accuracy']) 
sigma_values = np.linspace(0, 500, 11)
finite_state_values = np.linspace(0, 10, 11)
for fold in range(3):
    y_test = torch.load('fold_data/fold_%d_y_test.pt' % fold)
    for finite_states in finite_state_values:
        for sigma in sigma_values:
            patched_networks = []
            for model in range(4):
                net = torch.load('fold_%d_MLP_APS_Net_Model_%d.pt' % (fold, model))
                reference_memristor_params = {'time_series_resolution': 1e-6,
                                              'alpha_off': 1,
                                              'alpha_on': 3,
                                              'v_off': 0.5,
                                              'v_on': -0.53,
                                              'r_off': memtorch.bh.StochasticParameter(2.5e3, std=sigma*2, min=1),
                                              'r_on': memtorch.bh.StochasticParameter(100, std=sigma, min=1),
                                              'k_off': 4.03e-8,
                                              'k_on': -80,
                                              'd': 10e-9,
                                              'x_on': 0,
                                              'x_off': 10e-9}
                patched_networks.append(patch_model(net,
                                                  memristor_model=reference_memristor,
                                                  memristor_model_params=reference_memristor_params,
                                                  module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                                                  mapping_routine=naive_map,
                                                  transistor=True,
                                                  programming_routine=None))
                if finite_states != 0:
                    patched_networks[model] = apply_nonidealities(patched_networks[model],
                                                      non_idealities=[memtorch.bh.nonideality.NonIdeality.FiniteConductanceStates],
                                                      conductance_states = int(finite_states))
                patched_networks[model].tune_()
                
            concat_test_output = torch.zeros(y_test.numel(), 5).to(device) 
            for network_idx, network in enumerate(patched_networks):
                inputs = torch.load('fold_data/fold_%d_x_concat_test.pt' % (fold))[network_idx]
                for input_idx, input_ in enumerate(inputs):
                    concat_test_output[input_idx, :] += network(torch.tensor(input_).unsqueeze(0).to(device)).squeeze()
        
            concat_test_output_ = torch.argmax(concat_test_output, dim=1)
            correct = concat_test_output_.eq(y_test.to(device).data.view_as(concat_test_output_)).cpu().sum()
            test_set_accuracy = 100. * float(correct) / float(y_test.numel())
            df = df.append({'network_type': 'MLP', 'input': 'APS', 'finite_states': finite_states, 'sigma': sigma, 'fold': fold, 'test_set_accuracy': test_set_accuracy}, ignore_index=True)
            
networks = {}
networks['MLP_EMG_Net.pt'] = [MLP_EMG_Net, 'x_emg_train', 'x_emg_test', False]
networks['MLP_Fused_Net.pt'] = [MLP_Fused_Net, 'x_fused_train_mlp', 'x_fused_test_mlp', False]
networks['Conv_APS_Net.pt'] = [Conv_APS_Net, 'x_aps_train', 'x_aps_test', True]
networks['Conv_EMG_Net.pt'] = [Conv_EMG_Net, 'x_emg_train', 'x_emg_test', False]
networks['Conv_Fused_Net.pt'] = [Conv_Fused_Net, 'x_fused_train_conv', 'x_fused_test_conv', False]
for finite_states in finite_state_values:
    for sigma in sigma_values:
        for fold in range(3):
            y_test = torch.load('fold_data/fold_%d_y_test.pt' % fold)
            for network in networks:
                net = torch.load('fold_%d_%s' % (fold, network)).to(device)
                reference_memristor_params = {'time_series_resolution': 1e-6,
                                              'alpha_off': 1,
                                              'alpha_on': 3,
                                              'v_off': 0.5,
                                              'v_on': -0.53,
                                              'r_off': memtorch.bh.StochasticParameter(2.5e3, std=sigma*2, min=1),
                                              'r_on': memtorch.bh.StochasticParameter(100, std=sigma, min=1),
                                              'k_off': 4.03e-8,
                                              'k_on': -80,
                                              'd': 10e-9,
                                              'x_on': 0,
                                              'x_off': 10e-9}
                patched_net = patch_model(copy.deepcopy(net),
                                          memristor_model=reference_memristor,
                                          memristor_model_params=reference_memristor_params,
                                          module_parameters_to_patch=[torch.nn.Linear, torch.nn.Conv2d],
                                          mapping_routine=naive_map,
                                          transistor=True,
                                          programming_routine=None)
                
                del net
                if finite_states != 0:
                    patched_net = apply_nonidealities(copy.deepcopy(patched_net),
                                                      non_idealities=[memtorch.bh.nonideality.NonIdeality.FiniteConductanceStates],
                                                      conductance_states = int(finite_states))
                patched_net.tune_()
                test_loader = DataLoader(APS(torch.load('fold_data/fold_%d_%s.pt' % (fold, networks[network][2])), 
                                         y_test, 
                                         expand_dim=networks[network][3]), batch_size=batch_size, shuffle=False)
                test_set_accuracy = test(patched_net, test_loader)
                df = df.append({'network_type': det_network_type(network), 'input': det_input_type(network), 'finite_states': finite_states, 'sigma': sigma, 'fold': fold, 'test_set_accuracy': test_set_accuracy}, ignore_index=True)

df.to_csv('Variability_Finite_States.csv', index=False)  
df = df.sort_values(['network_type','input', 'finite_states', 'sigma', 'fold'])
df['test_set_accuracy'] = pd.to_numeric(df['test_set_accuracy'])
frame = df.groupby(['network_type', 'input', 'finite_states', 'sigma'])['test_set_accuracy']
df['test_set_accuracy_mean'] = frame.transform(np.mean)
df['test_set_accuracy_std'] = frame.transform(np.std)
df = df.drop(columns=['test_set_accuracy'])
df = df.drop_duplicates(subset=['network_type', 'input', 'finite_states', 'sigma', 'test_set_accuracy_mean', 'test_set_accuracy_std'], keep='first')
df = df.drop(columns=['fold'])
print(df)